<a href="https://colab.research.google.com/github/ariegever/ImageProcessing_Project/blob/main/5_unet_makingraster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === HOW TO USE THIS NOTEBOOK ===
#
# 1.  **Set Runtime:** Ensure you are using a GPU runtime (T4).
#
# 2.  **Configuration:**
#     * Configuration is now handled in `config.py`.
#
# 3.  **Run All Cells:**
#     * This notebook will:
#         a. Load your trained model.
#         b. Export a large image from GEE to Google Drive.
#         c. Read that image back from Drive.
#         d. Run prediction using smooth windowing (no external library needed).
#         e. Save the final classification map.

In [ ]:
from google.colab import drive
import config
drive.mount(config.DRIVE_MOUNT_PATH)
from google.colab import auth
import google.auth
import ee
# Trigger the authentication flow.
auth.authenticate_user()
# Get credentials and initialize Earth Engine
credentials, project = google.auth.default()
ee.Initialize(credentials, project=config.PROJECT_ID, opt_url='https://earthengine-highvolume.googleapis.com')

print(f"Successfully initialized Earth Engine for project: {config.PROJECT_ID}")

!pip install patchify rasterio scikit-image
# Note: smooth-tiled-predictions-plus is NOT needed anymore. We use utils.py.

import os
import json
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import tensorflow as tf
from tensorflow import keras
from matplotlib.colors import ListedColormap
import utils
from patchify import patchify, unpatchify

In [ ]:
# === Configuration from config.py ===

MODEL_PATH = os.path.join(config.DRIVE_IMAGES_PATH, config.MODEL_FILENAME)
EXPORT_FOLDER_NAME = "GEE_Exports_For_project" # Folder name in Drive Root
EXPORT_PATH_LOCAL = os.path.join(config.DRIVE_MOUNT_PATH, "MyDrive", EXPORT_FOLDER_NAME)

print(f"Model Path: {MODEL_PATH}")
print(f"Export Folder (Drive): {EXPORT_FOLDER_NAME}")
print(f"Export Path (Local): {EXPORT_PATH_LOCAL}")

In [ ]:
# 1. Load the Trained Model
try:
    model = keras.models.load_model(MODEL_PATH)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# 2. Load Class Definitions (for colormap)
import pandas as pd
with open(config.CLASS_JSON_PATH) as f:
    lc = json.load(f)
lc_df = pd.DataFrame.from_dict(lc, orient='index')
lc_df = lc_df.rename(columns={'class': 'label', 'color': 'palette'})
palette_hex = lc_df["palette"].to_list()
cmap = ListedColormap(palette_hex)
NUM_CLASSES = len(lc_df) + 1
print(f"Classes: {len(lc_df)}")

In [ ]:
# --- Export Image from GEE to Drive ---
# This step exports the image we want to classify.

# 1. Load Assets
s2_image = ee.Image(config.S2_ASSET_ID).select(config.S2_BANDS)
s1_image = ee.Image(config.S1_ASSET_ID).select(config.S1_BANDS)
feature_image = s2_image.addBands(s1_image)

# 2. Define Region (using the S2 image geometry)
geometry = s2_image.geometry()
scale = config.SCALE

# 3. Define Export Task
image_export_name = f"{config.AOI_NAME}_stacked_{config.YEAR}"
task = ee.batch.Export.image.toDrive(
    image=feature_image,
    description=image_export_name,
    folder=EXPORT_FOLDER_NAME, # This puts it in the correct folder!
    region=geometry,
    scale=scale,
    crs='EPSG:4326',
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()

print(f"Export task '{image_export_name}' started.")
print(f"It will be saved to Drive folder: '{EXPORT_FOLDER_NAME}'")
print("Please wait for the task to complete in GEE before proceeding.")
print("You can check status at: https://code.earthengine.google.com/tasks")

In [ ]:
# --- Load the Exported Image ---
# Wait for the export to finish before running this!

tif_filename = f"{image_export_name}.tif"
tif_path = os.path.join(EXPORT_PATH_LOCAL, tif_filename)

if not os.path.exists(tif_path):
    print(f"WAIT! File not found yet: {tif_path}")
    print("If the GEE task is finished, make sure the file is in the correct Drive folder.")
else:
    print(f"Found image: {tif_path}")
    
    with rasterio.open(tif_path) as src:
        large_image = src.read()
        # Transpose to (H, W, C) for model
        large_image = np.moveaxis(large_image, 0, -1)
        profile = src.profile
    
    print(f"Image shape: {large_image.shape}")
    
    # Handle NaNs
    large_image = np.nan_to_num(large_image, nan=0.0)
    
    # Clip to 0-1 (matching training data)
    large_image = np.clip(large_image, 0.0, 1.0)
    
    # --- Prediction ---
    print("Starting prediction with smooth windowing...")
    # Using the function from utils.py
    predictions_smooth = utils.predict_img_with_smooth_windowing(
        large_image,
        window_size=config.PATCH_SIZE,
        subdivisions=2,
        nb_classes=NUM_CLASSES,
        pred_func=model.predict
    )
    
    final_prediction = np.argmax(predictions_smooth, axis=-1)
    
    # --- Save Result ---
    output_filename = f"{config.AOI_NAME}_classification_{config.YEAR}.tif"
    output_path = os.path.join(EXPORT_PATH_LOCAL, output_filename)
    
    # Update profile for single band output
    profile.update(count=1, dtype=rasterio.uint8)
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(final_prediction.astype(rasterio.uint8), 1)
        
    print(f"Classification saved to: {output_path}")
    
    # --- Visualize ---
    plt.figure(figsize=(10, 10))
    plt.imshow(final_prediction, cmap=cmap, vmin=0, vmax=NUM_CLASSES)
    plt.title("Land Cover Classification")
    plt.axis('off')
    plt.show()